### Initialisation

In [1]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


### Récupération des données

In [2]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Création de la matrice des valeurs

In [3]:
function creerMatrice(price, weight, capacity)
n = length(price)
C = zeros(n,capacity+1)
for j in 2:capacity+1
    if j > weight[1]
        C[1,j] = max(C[1,j-1],C[1,j-weight[1]] + price[1])
    else
        C[1,j] = C[1,j-1]
    end
end
for i in 2:n
    for j in 1:capacity+1
        if j > weight[i]
            C[i,j] = max(C[i-1,j],C[i-1,j-weight[i]] + price[i])
        
        else
            C[i,j] = C[i-1,j]
        end
    end
end
println(C)
return C
end



creerMatrice (generic function with 1 method)

In [4]:
function solution(price, weight, capacity)
    l = []
    capacity2 = capacity
    i = length(price)
    C = creerMatrice(price, weight, capacity) 
    while i > 1
        if (C[i,capacity] != C[i-1,capacity])
            push!(l,i)
            capacity -= weight[i-1]
        end
        i -= 1
    end
    println(length(price))

    return l,C[length(price),capacity2]
end
#solution([42,40,12,25],[7,4,3,5],10)

solution (generic function with 1 method)

### Programme principal

In [5]:
function SolveKnapInstance(filename)

    price, weight, capacity = readKnaptxtInstance(filename)
    tableau = creerMatrice(price, weight, capacity)
    l, BestProfit = solution(price, weight, capacity)
    BestSol = zeros(length(price))
    for i in l
        BestSol[i] = 1
    end
    return BestProfit, BestSol

end


SolveKnapInstance (generic function with 1 method)

### Affichage du résultat

In [6]:
BestProfit, BestSol = SolveKnapInstance("instancesETU/KNAPnewformat/test.opb.txt")
println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", BestSol)

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 42.0 42.0 42.0 42.0; 0.0 0.0 0.0 0.0 40.0 40.0 40.0 42.0 42.0 42.0 42.0; 0.0 0.0 0.0 12.0 40.0 40.0 40.0 52.0 52.0 52.0 54.0; 0.0 0.0 0.0 12.0 40.0 40.0 40.0 52.0 52.0 65.0 65.0]
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 42.0 42.0 42.0 42.0; 0.0 0.0 0.0 0.0 40.0 40.0 40.0 42.0 42.0 42.0 42.0; 0.0 0.0 0.0 12.0 40.0 40.0 40.0 52.0 52.0 52.0 54.0; 0.0 0.0 0.0 12.0 40.0 40.0 40.0 52.0 52.0 65.0 65.0]


4

******

Optimal value = 65.0

Optimal x=

[0.0, 1.0, 0.0, 1.0]


### Réponses aux questions

La solution obtenue est adéquate car elle respecte les contraintes du problème : la somme des poids des objets sélectionnés dans la solution ne dépasse pas la capacité maximale du sac à dos et les objets sélectionnés dans la solution ont bien été pris en compte dans le calcul de la valeur finale (BestProfit). C'est la même solution qu'on a obtenu avec le branch and bound dans le TP2.

Dans cet algorithme, on crée une matrice n*(capacity + 1) où n est le nombre d'objet. Les colonnes de la matrice représentent la capacité restante de 0 à capacity. Pour chaque élément de la matrice, on vérifie si l'objet correspondant à cette ligne peut être pris ou non. Si l'objet peut être pris, on calcule la valeur obtenue en prenant cet objet ou pas, et choisit la valeur la plus grande. Sinon, on prend la valeur de la ligne précédente. Ensuite, on effectue un retour inverse de la matrice et on vérifie pour chaque objet si il a été pris ou pas; si oui on ajoute son indice dans une liste.

### Tests

In [7]:
BestProfit1, BestSol1 = SolveKnapInstance("instancesETU/KNAPnewformat/circle/knapPI_16_20_1000_1_-2291.opb.txt")
println("\n******\n\nOptimal value = ", BestProfit1, "\n\nOptimal x=", BestSol1)

BestProfit2, BestSol2 = SolveKnapInstance("instancesETU/KNAPnewformat/multiple_strongly_correlated/knapPI_14_20_1000_1_-1794.opb.txt")
println("\n******\n\nOptimal value = ", BestProfit2, "\n\nOptimal x=", BestSol2)

BestProfit3, BestSol3 = SolveKnapInstance("instancesETU/KNAPnewformat/strongly_correlated_span/knapPI_13_20_1000_1_-1716.opb.txt")
println("\n******\n\nOptimal value = ", BestProfit3, "\n\nOptimal x=", BestSol3)

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 403.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0

 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2418.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.

 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 806.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0

 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 2821.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3224.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.0 3627.

 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1209.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 1612.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.0 2015.

...

 2853.0 2853.0 2853.0 2853.0 2853.0 2853.0 2853.0 2853.0 2940.0 2940.0 2940.0 2940.0 2940.0 2940.0 2940.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0 3147.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 294.0 2

 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1677.0 1764.0 1764.0 1764.0 1764.0 1764.0 1764.0 1764.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.

 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 207.0 294.0 294.0 294.0 294.0 294.0 294.0 294.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501.0 501

Excessive output truncated after 524289 bytes.

 1677.0 1764.0 1764.0 1764.0 1764.0 1764.0 1764.0 1764.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 1971.0 2058.0 2058.0 2058.0 2058.0 2058.0 2058.0 2058.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.0 2265.

(Voir les tests réalisées avec branch and bound à la fin du notebook KnapsackNEW_ETU)

On en déduit que l'algorithme de programmation dynamique est plus performant en terme de temps d'exécution, mais l'algorithme du branch and bound reste plus utile lorsque la taille du problème est trop importante (Pour des problèmes de grande taille, l'algorithme de programmation dynamique retourne parfois "Excessive output").